# 종합점수.csv 수정

In [ ]:
import pandas as pd
import json

# CSV 파일을 올바른 헤더로 불러오기
# skiprows=1을 사용해서 첫 번째 행을 건너뛰고, 두 번째 행을 헤더로 사용
df1 = pd.read_csv("../data/raw/시군별_논벼_생산량_정곡.csv", skiprows=1)
df2 = pd.read_csv("../data/raw/시군별_논벼_생산량_조곡.csv", skiprows=1)
df3 = pd.read_csv("../data/raw/시군별_논벼_생산량_현미.csv", skiprows=1)

# 데이터 확인
print("df1 칼럼:", df1.columns.tolist())
print("df1 head:")
print(df1.head())
print()

df1 칼럼: ['행정구역별(1)', '행정구역별(2)', '논벼:재배면적 (ha)', '논벼:10a당 생산량 (kg)', '논벼:생산량 (톤)']
df1 head:
  행정구역별(1) 행정구역별(2)  논벼:재배면적 (ha)  논벼:10a당 생산량 (kg)  논벼:생산량 (톤)
0       전국       소계        697684               514     3584543
1    서울특별시       소계           184               498         918
2    부산광역시       소계          1940               453        8790
3    대구광역시       소계          5196               501       26056
4    인천광역시       소계         11214               503       56375



In [120]:
# 불필요한 데이터 필터링 (전국, 소계 제거)
def filter_data(df):
    # 전국 제거
    df = df[df['행정구역별(1)'] != '전국']
    
    # 도 단위 소계 제거 (전남, 전북, 경남, 경북, 충남, 충북, 강원도, 경기도의 소계)
    도_단위 = ['전라남도', '전라북도', '전북특별자치도', '경상남도', '경상북도', 
              '충청남도', '충청북도', '강원도', '강원특별자치도', '경기도']
    
    # 도 단위이면서 소계인 행 제거
    mask = ~((df['행정구역별(1)'].isin(도_단위)) & (df['행정구역별(2)'] == '소계'))
    df = df[mask]
    
    return df

df1 = filter_data(df1)
df2 = filter_data(df2)
df3 = filter_data(df3)

# 지역명 변경 함수
def create_region_name(row):
    region1 = row['행정구역별(1)']
    region2 = row['행정구역별(2)']
    
    # "소계" 단어 제거
    if region2 == '소계':
        region2 = ''
    
    # 특별한 경우들 처리
    if region1 == '세종특별자치시':
        return '세종시'
    elif region1 == '제주특별자치도' or region1 == "제주도":
        return region2 if region2 else '제주시'
    
    # 일반적인 도/시 변경
    region_mapping = {
        '강원특별자치도': '강원',
        '강원도': '강원',
        '경기도': '경기',
        '경상남도': '경남',
        '경상북도': '경북',
        '전라남도': '전남',
        '전라북도': '전북',
        '전북특별자치도': '전북',
        '충청남도': '충남',
        '충청북도': '충북'
    }
    
    # 광역시/특별시의 경우
    if '특별시' in region1 or '광역시' in region1:
        if region2:  # region2가 있으면
            return region1 + ' ' + region2
        else:  # region2가 없으면 (소계였던 경우)
            return region1
    
    # 도는 줄임말로 변경
    if region1 in region_mapping:
        if region2:  # region2가 있으면
            return region_mapping[region1] + ' ' + region2
        else:  # region2가 없으면
            return region_mapping[region1]
    
    # 기타의 경우
    if region2:
        return region1 + ' ' + region2
    else:
        return region1

# 지역 칼럼 생성
df1['지역'] = df1.apply(create_region_name, axis=1)
df2['지역'] = df2.apply(create_region_name, axis=1)
df3['지역'] = df3.apply(create_region_name, axis=1)

In [117]:
# 10a당 생산량 칼럼 추출 (칼럼명이 정확하지 않을 수 있으므로 확인 필요)
# 실제 칼럼명을 확인한 후 아래 칼럼명을 수정해주세요
df1_selected = df1[['지역', '논벼:10a당 생산량 (kg)']].rename(columns={'논벼:10a당 생산량 (kg)': '10a당_생산량_kg_정곡'})
df2_selected = df2[['지역', '논벼:10a당 생산량 (kg)']].rename(columns={'논벼:10a당 생산량 (kg)': '10a당_생산량_kg_조곡'})
df3_selected = df3[['지역', '논벼:10a당 생산량 (kg)']].rename(columns={'논벼:10a당 생산량 (kg)': '10a당_생산량_kg_현미'})

# 데이터 타입 확인 및 변환
for df_sel in [df1_selected, df2_selected, df3_selected]:
    for col in df_sel.columns:
        if '10a당_생산량_kg' in col:
            df_sel[col] = pd.to_numeric(df_sel[col], errors='coerce')

# 지역별로 조인
merged_df = df1_selected.merge(df2_selected, on='지역', how='outer')
merged_df = merged_df.merge(df3_selected, on='지역', how='outer')

# 10a당 생산량 합계 계산
merged_df['경작지 당 생산량 [kg/ha]'] = (
    (merged_df['10a당_생산량_kg_정곡'] + 
    merged_df['10a당_생산량_kg_조곡'] + 
    merged_df['10a당_생산량_kg_현미']) / 10
)

In [118]:
# 최종 결과 테이블 (지역과 합계만)
result_df = merged_df[['지역', '경작지 당 생산량 [kg/ha]']]

print("최종 결과:")
print(result_df.head(10))

# DataFrame을 딕셔너리로 변환
result_dict = result_df.to_dict('records')

# JSON 파일로 저장 (ensure_ascii=False로 한글 유지)
with open('../docs/data/논벼_경작지당_생산량.json', 'w', encoding='utf-8') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=2)

result_df.to_csv('../data/논벼_경작지당_생산량.csv', index=False, encoding='utf-8')

print("\n결과 미리보기:")
print(result_df.head(10))

최종 결과:
       지역  경작지 당 생산량 [kg/ha]
0  강원 강릉시              179.4
1  강원 고성군              172.1
2  강원 동해시              172.1
3  강원 삼척시              172.1
4  강원 속초시              171.9
5  강원 양구군              164.9
6  강원 양양군              171.9
7  강원 영월군              172.1
8  강원 원주시              178.0
9  강원 인제군              169.8

결과 미리보기:
       지역  경작지 당 생산량 [kg/ha]
0  강원 강릉시              179.4
1  강원 고성군              172.1
2  강원 동해시              172.1
3  강원 삼척시              172.1
4  강원 속초시              171.9
5  강원 양구군              164.9
6  강원 양양군              171.9
7  강원 영월군              172.1
8  강원 원주시              178.0
9  강원 인제군              169.8


In [119]:
# 경작지 당 생산량 값의 범위 확인
print("=== 경작지 당 생산량 [kg/10a] 통계 ===")
print(f"최솟값: {result_df['경작지 당 생산량 [kg/ha]'].min()}")
print(f"최댓값: {result_df['경작지 당 생산량 [kg/ha]'].max()}")
print(f"평균값: {result_df['경작지 당 생산량 [kg/ha]'].mean():.2f}")
print(f"중간값: {result_df['경작지 당 생산량 [kg/ha]'].median():.2f}")
print(f"표준편차: {result_df['경작지 당 생산량 [kg/ha]'].std():.2f}")

print("\n=== 기본 통계 요약 ===")
print(result_df['경작지 당 생산량 [kg/ha]'].describe())

=== 경작지 당 생산량 [kg/10a] 통계 ===
최솟값: 137.6
최댓값: 197.1
평균값: 171.77
중간값: 171.50
표준편차: 10.07

=== 기본 통계 요약 ===
count    157.000000
mean     171.773248
std       10.073082
min      137.600000
25%      165.000000
50%      171.500000
75%      177.400000
max      197.100000
Name: 경작지 당 생산량 [kg/ha], dtype: float64
